In [1]:
%run ../common/common.ipynb

Note: you may need to restart the kernel to use updated packages.


Caution: Nodes should have been already in `cluster.yaml`

In [12]:
run_command("kind delete cluster", shell=False)
run_command("kind create cluster --config=cluster.yaml", shell=False)
run_command("sleep 30", shell=False)

kind delete cluster
kind delete cluster
Log 0 - 2022-07-18 18:51:08 : Deleting cluster "kind" ...
kind create cluster --config=cluster.yaml
kind create cluster --config=cluster.yaml
Log 0 - 2022-07-18 18:51:11 : Creating cluster "kind" ...
Log 0 - 2022-07-18 18:51:11 : • Ensuring node image (kindest/node:v1.21.12) 🖼  ...
Log 0 - 2022-07-18 18:51:11 : ✓ Ensuring node image (kindest/node:v1.21.12) 🖼
Log 0 - 2022-07-18 18:51:11 : • Preparing nodes 📦 📦 📦 📦   ...
Log 0 - 2022-07-18 18:51:17 : ✓ Preparing nodes 📦 📦 📦 📦
Log 0 - 2022-07-18 18:51:17 : • Writing configuration 📜  ...
Log 0 - 2022-07-18 18:51:18 : ✓ Writing configuration 📜
Log 0 - 2022-07-18 18:51:18 : • Starting control-plane 🕹️  ...
Log 0 - 2022-07-18 18:51:29 : ✓ Starting control-plane 🕹️
Log 0 - 2022-07-18 18:51:29 : • Installing CNI 🔌  ...
Log 0 - 2022-07-18 18:51:31 : ✓ Installing CNI 🔌
Log 0 - 2022-07-18 18:51:31 : • Installing StorageClass 💾  ...
Log 0 - 2022-07-18 18:51:31 : ✓ Installing StorageClass 💾
Log 0 - 2022-07-18 

0

In [2]:
run_command('../common/common_modules.sh')
!kubectl apply -f https://raw.githubusercontent.com/kubernetes/ingress-nginx/main/deploy/static/provider/kind/deploy.yaml

../common/common_modules.sh
../common/common_modules.sh
Log 0 - 2025-01-14 10:55:36 : clusterrolebinding.rbac.authorization.k8s.io/manager-full created
Log 0 - 2025-01-14 10:55:38 : customresourcedefinition.apiextensions.k8s.io/certificaterequests.cert-manager.io created
Log 0 - 2025-01-14 10:55:38 : customresourcedefinition.apiextensions.k8s.io/certificates.cert-manager.io created
Log 0 - 2025-01-14 10:55:38 : customresourcedefinition.apiextensions.k8s.io/challenges.acme.cert-manager.io created
Log 0 - 2025-01-14 10:55:38 : customresourcedefinition.apiextensions.k8s.io/clusterissuers.cert-manager.io created
Log 0 - 2025-01-14 10:55:38 : customresourcedefinition.apiextensions.k8s.io/issuers.cert-manager.io created
Log 0 - 2025-01-14 10:55:39 : customresourcedefinition.apiextensions.k8s.io/orders.acme.cert-manager.io created
Log 0 - 2025-01-14 10:55:39 : namespace/cert-manager created
Log 0 - 2025-01-14 10:55:39 : serviceaccount/cert-manager-cainjector created
Log 0 - 2025-01-14 10:55:3

In [3]:
(manager_node, jobmanager_node, taskmanager_nodes) = get_label_nodes(ip_address=True)

kind-control-plane	{'beta.kubernetes.io/arch': 'amd64', 'beta.kubernetes.io/os': 'linux', 'kubernetes.io/arch': 'amd64', 'kubernetes.io/hostname': 'kind-control-plane', 'kubernetes.io/os': 'linux', 'node-role.kubernetes.io/control-plane': '', 'node-role.kubernetes.io/master': '', 'node.kubernetes.io/exclude-from-external-load-balancers': ''}
kind-worker	{'beta.kubernetes.io/arch': 'amd64', 'beta.kubernetes.io/os': 'linux', 'ingress-ready': 'true', 'kubernetes.io/arch': 'amd64', 'kubernetes.io/hostname': 'kind-worker', 'kubernetes.io/os': 'linux', 'tier': 'manager'}
kind-worker2	{'beta.kubernetes.io/arch': 'amd64', 'beta.kubernetes.io/os': 'linux', 'kubernetes.io/arch': 'amd64', 'kubernetes.io/hostname': 'kind-worker2', 'kubernetes.io/os': 'linux', 'tier': 'jobmanager'}
kind-worker3	{'beta.kubernetes.io/arch': 'amd64', 'beta.kubernetes.io/os': 'linux', 'kubernetes.io/arch': 'amd64', 'kubernetes.io/hostname': 'kind-worker3', 'kubernetes.io/os': 'linux', 'tier': 'taskmanager'}


In [4]:
!kubectl describe ing -n manager

Name:             grafana
Labels:           <none>
Namespace:        manager
Address:          localhost
Default backend:  default-http-backend:80 (<error: endpoints "default-http-backend" not found>)
Rules:
  Host                        Path  Backends
  ----                        ----  --------
  grafana.127-0-0-1.sslip.io  
                              /   prom-grafana:80 (10.244.1.5:3000)
Annotations:                  kubernetes.io/ingress.class: nginx
                              nginx.ingress.kubernetes.io/proxy-body-size: 0
Events:
  Type    Reason  Age                From                      Message
  ----    ------  ----               ----                      -------
  Normal  Sync    20s (x2 over 20s)  nginx-ingress-controller  Scheduled for sync


Name:             minio
Labels:           <none>
Namespace:        manager
Address:          localhost
Default backend:  default-http-backend:80 (<error: endpoints "default-http-backend" not found>)
Rules:
  Host               

In [10]:
address = "127-0-0-1"
print("Manager node: {}:30080".format(manager_node))
print("Access to Minio: http://minio.{}.nip.io:30080".format(address))
print("Access to Grafana: http://grafana.{}.nip.io:30080".format(address))
print("Access to Prometheus: http://prometheus.{}:30080".format(address))
print("Job manager address: {}".format(jobmanager_node))
print("Task manager addresses: {}".format(taskmanager_nodes))

Manager node: 172.23.0.3:30080
Access to Minio: http://minio.127-0-0-1.nip.io:30080
Access to Grafana: http://grafana.127-0-0-1.nip.io:30080
Access to Prometheus: http://prometheus.127-0-0-1:30080
Job manager address: 172.23.0.5
Task manager addresses: 172.23.0.2


In [11]:
print("Access to Minio: http://{}:30900".format(manager_node))
print("Access to Grafana: http://{}:30300".format(manager_node))
print("Access to Prometheus: http://{}:30090".format(manager_node))
print("Job manager address: {}".format(jobmanager_node))
print("Task manager addresses: {}".format(taskmanager_nodes))

Access to Minio: http://172.23.0.3:30900
Access to Grafana: http://172.23.0.3:30300
Access to Prometheus: http://172.23.0.3:30090
Job manager address: 172.23.0.5
Task manager addresses: 172.23.0.2


Import [Flink dashboard](https://grafana.com/grafana/dashboards/14911) in Grafana.